In [1]:
import matplotlib.pyplot as plt
import numpy as np
import Nio,os
from libtiff import TIFF 
from osgeo import gdal
import Ngl
from pandas import DataFrame
import pandas as pd
def find_pwvs(pwv,arrlat,arrlon,coor):
    latc = np.abs(arrlat-coor[0])
    lat_row,lat_li=find_min_pos(latc)
    lonc = np.abs(arrlon-coor[1])
    lon_row,lon_li=find_min_pos(lonc)
#     print(lat_row,lon_li)
    return pwv[lat_row][lon_li]
def find_bd(data,arrlat,arrlon,coor):
    latc = np.abs(arrlat-coor[0])
    lat_row,lat_li=find_min_pos(latc)
    lonc = np.abs(arrlon-coor[1])
    lon_row,lon_li=find_min_pos(lonc)
#     print(lat_row,lon_li)
    return data[:,lat_row,lon_li]
def find_min_pos(arr):
    c = np.abs(arr-arr.min())
    row = (np.argmin(c))//(c.shape[1])
    li = (np.argmin(c))%(c.shape[1])
    return row,li

def GetCoor(dataset):
#     print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
#                                 dataset.GetDriver().LongName))
#     print("Size is {} x {} x {}".format(dataset.RasterXSize,
#                                         dataset.RasterYSize,
#                                         dataset.RasterCount))
#     print("Projection is {}".format(dataset.GetProjection()))
    geotransform = dataset.GetGeoTransform()
    if geotransform:
#         print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
#         print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))
        cor = [[geotransform[0], geotransform[3]],[geotransform[0]+geotransform[1]*dataset.RasterXSize,geotransform[3]+geotransform[5]*dataset.RasterYSize]]
#         print(geotransform[0]+geotransform[1]*dataset.RasterXSize,geotransform[3]+geotransform[5]*dataset.RasterYSize)
#         print(dataset.ReadAsArray().shape)
        lon = np.arange(0,dataset.RasterXSize)
        lat = np.arange(0,dataset.RasterYSize)
        lat = lat*geotransform[5]+geotransform[3]
        lat = np.tile(lat,dataset.RasterXSize)
        lat = lat.reshape(dataset.RasterXSize,dataset.RasterYSize)
        lat = lat.T
        lon = lon*geotransform[1]+geotransform[0]
        lon = np.tile(lon,dataset.RasterYSize).reshape(dataset.RasterYSize,dataset.RasterXSize)
#         print(lon)
        return lat,lon
    else:
        print("No Geotransform")
def Get_PWV(date,time,coor):
    path = "/data/wx_data/2008_pwv_tiff/"
#     time = time.split("T")[1]
    time = time.split(":")
    h = time[0]
    m = time[1]
    upperf = path+"2008"+date.split("-")[1]+date.split("-")[2]+h+"_pwv.tif"
    lowerf = path+"2008"+date.split("-")[1]+date.split("-")[2]+str(int(h)+1).zfill(2)+"_pwv.tif"
    datasetu = gdal.Open(upperf)
    lat ,lon = GetCoor(datasetu)
    pwvu = find_pwvs(datasetu.ReadAsArray(),lat,lon,coor)
    datasetl = gdal.Open(lowerf)
    lat ,lon = GetCoor(datasetl)
    pwvl = find_pwvs(datasetl.ReadAsArray(),lat,lon,coor)
    pwv = (pwvl-pwvu)*int(m)/60+pwvu
#     print(pwv)
    return pwv
def Get_PWV_Array(date,time,dataset):
    latarr,lonarr = GetCoor(dataset)
    pwvarr = np.zeros_like(latarr)
    path = "/data/wx_data/2008_pwv_tiff/"
#     time = time.split("T")[1]
    time = time.split(":")
    h = time[0]
    m = time[1]
    upperf = path+"2008"+date.split("-")[1]+date.split("-")[2]+h+"_pwv.tif"
    lowerf = path+"2008"+date.split("-")[1]+date.split("-")[2]+str(int(h)+1).zfill(2)+"_pwv.tif"
    datasetu = gdal.Open(upperf)
    datasetl = gdal.Open(lowerf)
    lat ,lon = GetCoor(datasetu)
    for i in range(latarr.shape[0]):
        for j in range(latarr.shape[1]):
            coor=[latarr[i][j],lonarr[i][j]]
            pwvu = find_pwvs(datasetu.ReadAsArray(),lat,lon,coor)
            pwvl = find_pwvs(datasetl.ReadAsArray(),lat,lon,coor)
            pwvarr[i][j] = (pwvl-pwvu)*int(m)/60+pwvu
#             print(j)
    print(date,h,m,"PWV Calculated")
    return pwvarr
def Calc(dataset,pwv,para):
    imga = dataset.ReadAsArray()
    imga = np.array(imga,dtype='float64')
    imga[imga==0] = np.nan
#     B3AP = float(para[0])
#     B3MP = float(para[1]) 
#     B4AP = float(para[2])
#     B4MP = float(para[3])
#     B3D = np.flipud(imga[0])
#     B4D = np.flipud(imga[1])
#     B6D = np.flipud(imga[2])
#     B3R = B3D*B3MP+B3AP
#     B4R = B4D*B4MP+B4AP
#     B6R = (B6D-1)*(12.65-3.2)/254+3.2
    B3R = np.flipud(imga[0])*0.0001
    B4R = np.flipud(imga[1])*0.0001
    B6 = np.flipud(imga[2])*0.1
    NDVI = (B4R-B3R)/(B4R+B3R)
    LSE = NDVI
    LSE[LSE>0.5]=0.985
    LSE[LSE<0.1] = 0.96
    IG = 0
    IV = np.max(NDVI)
    I = NDVI 
    PV = (I-0.2)**2/0.09
    t = (LSE<0.5)&(LSE>0.1)
    t = np.array(t)
    LSE[t]= 0
    PVS = 0.985*PV+0.96*(1-PV)+(1-0.96)*(1-PV)*0.985*0.55
    PVS[~t] = 0
    LSE = LSE + PVS
    TS = B6
    B6R = 666.09/(np.exp(1282.71/TS))-1
#     TS = 1282.71/np.log((666.09/B6R+1))
    GAM = 1/(14387*B6R/TS**2*(11.45**4*B6R/(1.19104*10**8)+1/11.45))
    DEL = -GAM*B6R+TS
    w = pwv
    F1 = 0.09172*w**2-0.09894*w+1.09659
    F2 = -0.71656*w**2-0.64218*w-0.17183
    F3 = -0.03503*w**2+1.54063*w-0.46434
    TSC = GAM*((F1*B6R+F2)/LSE+F3)+DEL
#     plt.contourf(TSC,cmap='gray')
#     plt.colorbar()
    return TSC
def CalcS(dataset,pwv):
#     B3AP = float(para[0])
#     B3MP = float(para[1]) 
#     B4AP = float(para[2])
#     B4MP = float(para[3])
    B3R = dataset[0]*0.0001
    B4R = dataset[1]*0.0001
    B6T = dataset[2]*0.1
#     print(B3R,B4R,B6T,pwv)
#     B3R = B3D*B3MP+B3AP
#     B4R = B4D*B4MP+B4AP
#     B6R = (B6D-1)*(12.65-3.2)/254+3.2
    NDVI = (B4R-B3R)/(B4R+B3R)
    LSE = NDVI
    if(NDVI>0.5):
        LSE = 0.985
    elif(NDVI<0.1):
        LSE = 0.96
    else:
        I = NDVI 
        PV = (I-0.2)**2/0.09
        PVS = 0.985*PV+0.96*(1-PV)+(1-0.96)*(1-PV)*0.985*0.55
        LSE = PVS
#     TS = 1282.71/np.log((666.09/B6R+1))
#     print(LSE)
    TS = B6T
    B6R = 666.09/(np.exp(1282.71/TS))-1
    GAM = 1/(14387*B6R/TS**2*(11.45**4*B6R/(1.19104*10**8)+1/11.45))
    DEL = -GAM*B6R+TS
    w = pwv
    F1 = 0.09172*w**2-0.09894*w+1.09659
    F2 = -0.71656*w**2-0.64218*w-0.17183
    F3 = -0.03503*w**2+1.54063*w-0.46434
    TSC = GAM*((F1*B6R+F2)/LSE+F3)+DEL
#     plt.contourf(TSC,cmap='gray')
#     plt.colorbar()
    return TSC
def DrawByNgl(nplot,lat,lon,sitename):
    boxlat1 = np.array([33.06429,29.7645,
             30.7750,28.3581,31.926])
    boxlon1 = np.array([91.94256,94.7384,
                 90.9885,86.9464,91.716])
    txt1 = np.array(["D105","Linzh","NamCo","QOMO","MS3478"])


    site_info = DataFrame(data=[boxlat1,boxlon1],index=["Lat","Lon"],columns=txt1)
    shppath = r'/data/wx_data/Tibet_shape/qingzang.shp'
    shpf = Nio.open_file(shppath, "r")
    shp_lon = shpf.variables["x"][:]
    shp_lat = shpf.variables["y"][:]
    min_shp_lat = min(shp_lat)
    max_shp_lat = max(shp_lat)
    min_shp_lon = min(shp_lon)
    max_shp_lon = max(shp_lon)
    print(min_shp_lat,max_shp_lat,min_shp_lon,max_shp_lon)
    ws_id  = Ngl.get_workspace_id()
    srlist = Ngl.Resources()
    srlist.wsMaximumSize = 1000000000
    Ngl.set_values(ws_id,srlist)
    rlist = Ngl.Resources()
    wks_type = "png"
    rlist.wkWidth = 3000
    rlist.wkHeight = 1000
    pngname = "/home/wx/Jupyter_script/jupyter/LST"+sitename
    wks = Ngl.open_wks(wks_type,pngname,rlist)
    cnres = Ngl.Resources()         
    cnres.gsnScalarContour  = True
    cnres.nglDraw           = False
    cnres.nglFrame          = False
    cnres.cnFillOn          = True     
    cnres.cnLinesOn         = False         # turn off contour lines
    cnres.cnLineLabelsOn    = False         # turn off line labels
    cnres.cnFillPalette     = "temp_diff_18lev"
#     cnres.cnFillPalette     = "MPL_BrBG"
    # cnres.cnLevelSelectionMode="ExplicitLevels"
    # cnres.cnLevelSpacingF   = 0.05              # contour level spacing  # can speed up plotting.
    # cnres.cnLevels          = np.linspace(0,1,10)
    cnres.cnLevelSelectionMode="ManualLevels"
    cnres.tmXBLabelFontHeightF   = 0.035
    cnres.tmYLLabelFontHeightF   = 0.035
    cnres.cnMaxLevelValF        = 320
    cnres.cnMinLevelValF        = 240
    cnres.cnLevelSpacingF       = 5
    cnres.cnFillMode        = "RasterFill"        # These two resources
    cnres.mpDataBaseVersion     = "MediumRes"
    cnres.sfMissingValueV   = -999
    cnres.mpOutlineOn           = True         # Turn on map outlines
    cnres.mpOutlineBoundarySets = "National"
    # cnres.mpProjection = 'Mercator'
    cnres.mpLimitMode       = "LatLon"
    cnres.mpPerimOn       = True            # Turn on map perimeter.
    cnres.mpGridAndLimbOn = False           # Turn off map grid.
    cnres.pmLabelBarDisplayMode = "Never" 
    lnres                   = Ngl.Resources()
    lnres.gsLineColor       = "black"
    lnres.gsLineThicknessF  = 3.0            # 3x as thick
    data = []
    path = r'/data/wx_data/BYSJ/LST/'+sitename+"/"
    dirs = os.listdir(path)
    files = np.array(dirs)
    for i in files:
        fdata = np.loadtxt(path+i,dtype='float64',delimiter=',')
        data.append(Insertion(fdata))
    print(nplot)
    plots = []
    for i in range(0,nplot):
        cnres.sfXArray      = np.flipud(lon)
        cnres.sfYArray      = np.flipud(lat)
        cnres.mpMinLatF     = lat.min()
        cnres.mpMaxLatF     = lat.max()
        cnres.mpMinLonF     = lon.min()
        cnres.mpMaxLonF     = lon.max()
        plotdata = Fillna(data[i])
        plti = Ngl.contour_map(wks,plotdata,cnres)
        mkres = Ngl.Resources()
        mkres.gsMarkerIndex = 1
        mkres.gsMarkerSizeF = 23
        mkres.gsMarkerColor = "black"
        tres = Ngl.Resources()
        tres.txFontHeightF = 0.030
        tres.txFontColor   = "black"
        tres.txFont        = "helvetica-bold"
        x = site_info.iloc[1].values
        y = site_info.iloc[0].values
        for j in range(0,site_info.columns.size):
            text = site_info.columns.values[j]
            tx = x[j]+0.1
            ty = y[j]+0.1
#             Ngl.add_text(wks,plti,text,tx,ty,tres)
        Ngl.add_polymarker(wks,plti,x,y,mkres)
        Ngl.add_polyline(wks, plti, shp_lon, shp_lat, lnres)
        plots.append(plti)
    panelres                            = Ngl.Resources()
    panelres.nglPanelYWhiteSpacePercent = 3.
    panelres.nglPanelXWhiteSpacePercent = 3.
    panelres.nglPanelLabelBar                 = True     # Turn on panel labelbar
    panelres.nglPanelLabelBarLabelFontHeightF = 0.012    # Labelbar font height
    panelres.nglPanelLabelBarHeightF          = 0.0850   # Height of labelbar
    panelres.nglPanelLabelBarWidthF           = 0.700    # Width of labelbar
    panelres.lbLabelFont                      = "helvetica-bold" # Labelbar font
    panelres.lbBoxEndCapStyle                 = "TriangleBothEnds"
    panelres.nglPanelTop                      = 0.935
    panelres.nglPanelFigureStrings            = list(map(chr,range(65, 91)))
    panelres.nglPanelFigureStringsFontHeightF =  0.015
    panelres.nglPanelFigureStringsJust        = "BottomLeft"
    Ngl.panel(wks,plots[0:nplot],[2,nplot//2+1],panelres) 
    Ngl.end()
    print("Pannel Drew")
def Insertion(data):
    ex = pd.DataFrame(data)
    ex = ex.fillna(method='pad',limit=5)
    return ex.values
def Fillna(data):
    ex = pd.DataFrame(data)
    ex = ex.fillna(-999)
    return ex.values
def Get_BD(dataset,coors):
    lat ,lon = GetCoor(dataset)
    BD = find_bd(dataset.ReadAsArray(),lat,lon,coors)
    return BD
if __name__ =='__main__':
    sitename = 'MS3478R'
    coors = [31.22623,91.78328]
    path = r'/data/wx_data/BYSJ/'+sitename+'/'
    dirs = os.listdir(path)
    files = np.array(dirs)
    para = np.loadtxt(path+sitename+'.txt',dtype=bytes).astype(str) 
    nplot = 0
    n=0
    TSA = []
    for i in files:
        if (i.split('-')[0]=="2008"):
            dataf=path+i
            pwvf = path+'pwv/'+i.split('.')[0]+'pwv.tif'
            dataset = gdal.Open(dataf)
            lats,lons = GetCoor(dataset)
#             pwv = gdal.Open(pwvf)
#             pwvdata = pwv.ReadAsArray()
            date = i.split(".")[0]
            time = str(para[n]).split("T")[1]    
            PWVFP = "/data/wx_data/BYSJ/PWV/"+sitename+'/'+i.split(".")[0]+'pwv.txt'
            if not os.path.exists(PWVFP):
                A = Get_PWV_Array(date,time,dataset)
                np.savetxt(PWVFP, A,fmt='%f',delimiter=',')
            else:
                A = np.loadtxt(PWVFP,delimiter = ",",dtype = "float64")
            TS = Calc(dataset,A,para[n])
#             np.savetxt("/data/wx_data/BYSJ/LST/"+sitename+'/'+i.split(".")[0]+'.txt', TS,fmt='%f',delimiter=',')
#             print(i,"Saved")
#             print(time)
            pwvd = Get_PWV(date,time,coors)
#             print(A)
#             dataNamco = np.array([[2298,2269,2747],[2066,2253,2784],[3818,3136,2709],[3399,2684,2690],[0,1,0]])
#             dataqomo = 
#             BD = dataNamco[n]
            BD = Get_BD(dataset,coors)
            LSTS = CalcS(BD,pwvd)
            hour = time.split(":")[0]
            minu = time.split(":")[1]
            timem = hour+":"+minu
            print(date,timem,LSTS)
            n = n+1
            nplot = nplot +1
#     DrawByNgl(nplot,lats,lons,sitename)
#     print(TSA)
            

<ipython-input-1-8ac6870e3a4d>:113: RuntimeWarning: invalid value encountered in greater
  LSE[LSE>0.5]=0.985
<ipython-input-1-8ac6870e3a4d>:114: RuntimeWarning: invalid value encountered in less
  LSE[LSE<0.1] = 0.96
<ipython-input-1-8ac6870e3a4d>:119: RuntimeWarning: invalid value encountered in less
  t = (LSE<0.5)&(LSE>0.1)
<ipython-input-1-8ac6870e3a4d>:119: RuntimeWarning: invalid value encountered in greater
  t = (LSE<0.5)&(LSE>0.1)


2008-01-01 04:12 284.4053655582413
2008-01-17 04:12 282.74033441450933
2008-02-02 04:12 257.15732365588605
2008-11-16 04:11 267.9724306851495
2008-12-02 04:11 271.7770347537513
2008-12-18 04:11 281.6094363130884


In [50]:
import pandas as pd
sitenamesT = ['Namco','Linzhi','Qomo']
sitenamesTSFC = ['D105','MS3478L','MS3478R']
for sitename in sitenamesT:
    print(sitename)
    path = r'/data/wx_data/BYSJ/'+sitename+'/'
    para = np.loadtxt(path+sitename+'.txt',dtype=bytes).astype(str) 
    sitepath = r'/data/wx_data/BYSJ/sitedata/'
    sdata = pd.read_csv(sitepath+sitename+".csv")
    columns = sdata.columns
    sdata.columns=[x.strip() for x in columns if x.strip() != '']
#     print(sdata.columns)
    for i in para:
        date = str(i).split("T")[0]
        month = int(date.split("-")[1])
        day = int(date.split("-")[2])
        time = str(i).split("T")[1]
        hour = int(time.split(":")[0])
        minu = int(time.split(":")[1])
#         print(month,day,hour,minu)
        smdata = sdata[sdata['month'].isin([int(month)])]
        smddata = smdata[smdata['day'].isin([int(day)])]
        smdhdata = smddata[smddata['hour'].isin([int(hour+8)])]
        smdhdata.reset_index(drop=True, inplace=True)
        if(minu<30):
            LSTRU = smdhdata.loc[0,'Ts']
            LSTRL = smdhdata.loc[1,'Ts']
        else:
            LSTRU = smdhdata.loc[1,'Ts']
            smdhdata = smddata[smddata['hour'].isin([int(hour+9)])]
            smdhdata.reset_index(drop=True, inplace=True)       
            LSTRL = smdhdata.loc[0,'Ts']
        LST = LSTRU+(LSTRL-LSTRU)*minu/60 + 273.15
        print(LST)
#         LSTRU

Namco
275.95561666666663
-9726.75
-9726.75
-9726.75
276.6799
Linzhi
289.72799999999995
290.72965
288.4513833333333
284.0371666666666
Qomo
277.41839999999996
287.02099999999996
295.04966666666667
295.70059999999995
307.0169333333333
293.849
288.693
282.09751666666665
286.6400333333333


In [1]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
file = "/data/cml_data/"
os.listdir(file)




['.nohup.out.swp',
 'drought_indices',
 'sm',
 'TP_image',
 'public_use',
 'ESA_CCI_SMv4.4',
 'MOD11C1',
 'MOD09CMG',
 'ERA5',
 'MOD09GA',
 'MOD11A1',
 'atoms-chemis-phis',
 'cml',
 'MOD09GA_MOSAIC',
 'MOD11A1_MOSAIC']